# 패키지 불러오기

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb

from catboost import Pool, CatBoostRegressor
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression

n_splits = 5

# 물가상승률 계산

In [2]:
money_rate = pd.read_excel('../물가상승률데이터/연도별_소비자물가_등락률.xlsx')
money_rate = money_rate.loc[0].transpose().reset_index().loc[1:].reset_index(drop=True)
money_rate.columns = ['연도', '물가상승률']
money_rate


money_rate['물가상승률'] = money_rate['물가상승률'] * 0.01 + 1
money_rate.loc[0, '물가상승률'] = 1

c:\users\sungs\appdata\local\programs\python\python38\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
def mul(mul_list):
    
    answer = 1
    
    for num in mul_list:
        answer *= num
        
    return answer

for i in range(money_rate.shape[0]): # 1.48053
    money_rate.loc[i, '누적물가상승률'] = mul(money_rate.loc[:i, '물가상승률'])
    
money_rate['연도'] = money_rate['연도'].astype('int')
money_rate['물가상승률'] = money_rate['물가상승률'].astype('float')
money_rate['누적물가상승률'] = money_rate['누적물가상승률'].astype('float')

In [4]:
money_rate

,연도,물가상승률,누적물가상승률
0,2002,1.000,1.000000
1,2003,1.035,1.035000
2,2004,1.036,1.072260
3,2005,1.028,1.102283
4,2006,1.022,1.126534
5,2007,1.025,1.154697
6,2008,1.047,1.208968
7,2009,1.028,1.242819
8,2010,1.029,1.278860
9,2011,1.040,1.330015


# 연봉 변환

In [5]:
hitter = pd.read_csv('../선수데이터(전처리완료)/타자(모델링용).csv')
pitcher = pd.read_csv('../선수데이터(전처리완료)/투수(모델링용).csv')
hitter

,데뷔년도,연도,타율,경기,타석,타수,득점,안타,2루타,3루타,...,팀명_현대,포지션(수비)_1루수,포지션(수비)_2루수,포지션(수비)_3루수,포지션(수비)_수비기록없음,포지션(수비)_우익수,포지션(수비)_유격수,포지션(수비)_좌익수,포지션(수비)_중견수,포지션(수비)_포수
0,1997,2003,0.281,92,289,253,38,71,9,0,...,0,0,1,0,0,0,0,0,0,0
1,1997,2004,0.000,2,2,2,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,1997,2005,0.231,25,69,65,5,15,0,0,...,0,0,1,0,0,0,0,0,0,0
3,2019,2019,0.151,27,57,53,5,8,1,1,...,0,0,0,0,0,1,0,0,0,0
4,2019,2020,0.239,59,73,67,11,16,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4145,2005,2010,0.266,29,74,64,7,17,2,1,...,0,1,0,0,0,0,0,0,0,0
4146,2015,2020,0.000,5,4,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4147,2009,2009,0.286,15,26,21,4,6,1,0,...,0,0,0,0,0,0,0,0,0,1
4148,2009,2011,0.000,5,2,2,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [6]:
for i in range(hitter.shape[0]):
    
    year = hitter.loc[i, '연도']
    interest_rate = float(money_rate.loc[money_rate['연도'] == year, '누적물가상승률'])
    hitter.loc[i, '연봉'] = hitter.loc[i, '연봉'] / interest_rate


for i in range(pitcher.shape[0]):
    
    year = pitcher.loc[i, '연도']
    interest_rate = float(money_rate.loc[money_rate['연도'] == year, '누적물가상승률'])
    pitcher.loc[i, '연봉'] = pitcher.loc[i, '연봉'] / interest_rate

# 모델 정의
 1. Random Forest
 2. Lightgbm
 3. Xgboost
 4. Catboost
 5. Knn
 6. Linear regression

In [7]:
def Xy_split(dataset):
    
    X = dataset.drop('연봉', axis=1)
    column_dict = {f'variable{idx+1}':col for idx, col in enumerate(X.columns)}
    X = X.rename(columns = {col:f'variable{idx+1}' for idx, col in enumerate(X.columns)})
    y = dataset['연봉']
    
    return column_dict, X, y

In [8]:
def rf_model(data_x, data_y):

    # rf 모델링
    performance = []
    cv = KFold(n_splits = n_splits, shuffle = True, random_state=42)

    for tr_idx, val_idx in cv.split(data_x):  

        X_train = data_x.iloc[tr_idx, :]
        y_train = data_y[tr_idx]

        X_valid = data_x.iloc[val_idx, :]
        y_valid = data_y[val_idx]

        rf_model = RandomForestRegressor(random_state = 42)
        rf_model.fit(X_train, y_train)

        pred_valid = rf_model.predict(X_valid)
        rmse = mean_squared_error(y_valid, pred_valid, squared=False)
        performance.append(rmse)
        
    # rf 성능종합
    performance.append(np.mean(performance))

    output = pd.DataFrame({'rf':performance}, index=['cv1','cv2','cv3','cv4','cv5','평균'])
    return output

In [9]:
def lgbm_model(data_x, data_y):

    # lgbm 모델링
    lgb_params = {"objective" : "rmse",
                 "verbosity" : -1}

    lgb_models={}
    step = 1

    cv = KFold(n_splits = n_splits, shuffle = True, random_state=42)

    for tr_idx, val_idx in cv.split(data_x):  

        print(f'\n\n ============================ {step} ============================')    

        X_train = data_x.iloc[tr_idx, :].values
        y_train = data_y[tr_idx].values

        X_valid = data_x.iloc[val_idx, :].values
        y_valid = data_y[val_idx].values

        lgb_dtrain = lgb.Dataset(data = X_train, label = y_train) 
        lgb_dvalid = lgb.Dataset(data = X_valid, label = y_valid) 

        lgb_model = lgb.train(lgb_params, lgb_dtrain, 20000, valid_sets=[lgb_dvalid], early_stopping_rounds=100, verbose_eval=1000)
        lgb_models[step] = lgb_model

        step += 1
        
    # lgbm 성능종합
    performance = [lgb_models[step].best_score['valid_0']['rmse'] for step in range(1,6)]
    performance.append(np.mean(performance))

    output = pd.DataFrame({'lgbm':performance}, index=['cv1','cv2','cv3','cv4','cv5','평균'])
    return output

In [10]:
def xgb_model(data_x, data_y):
    
    # xgb 모델링
    xgb_final_param = {
          "objective" : 'reg:squarederror',
          "random_state" : 42,
          "verbosity" : 0
          }

    xgb_models={}
    step = 1

    cv = KFold(n_splits = n_splits, shuffle = True, random_state=42)

    for tr_idx, val_idx in cv.split(data_x):  

        X_train = data_x.iloc[tr_idx, :].values
        y_train = data_y[tr_idx].values

        X_valid = data_x.iloc[val_idx, :].values
        y_valid = data_y[val_idx].values

        xgb_dtrain = xgb.DMatrix(data = X_train, label = y_train) 
        xgb_dvalid = xgb.DMatrix(data = X_valid, label = y_valid) 

        xgb_model = xgb.train(params = xgb_final_param, dtrain = xgb_dtrain, num_boost_round = 20000, early_stopping_rounds = 100, verbose_eval = 1000, evals=[(xgb_dtrain, 'train'), (xgb_dvalid,'eval')])
        xgb_models[step] = xgb_model

        step += 1
        
    # xgb 성능종합
    performance = [xgb_models[step].best_score for step in range(1,6)]
    performance.append(np.mean(performance))

    output = pd.DataFrame({'xgb':performance}, index=['cv1','cv2','cv3','cv4','cv5','평균'])
    return output

In [11]:
def cb_model(data_x, data_y):

    # cb 모델링
    cat_cols = []

    cb_models={}
    step = 1

    cv = KFold(n_splits = n_splits, shuffle = True, random_state=42)

    for tr_idx, val_idx in cv.split(data_x):  

        X_train = data_x.iloc[tr_idx, :]
        y_train = data_y[tr_idx]

        X_valid = data_x.iloc[val_idx, :]
        y_valid = data_y[val_idx]

        cb_dtrain = Pool(data=X_train, label=y_train, cat_features=cat_cols)
        cb_dvalid = Pool(data=X_valid, label=y_valid, cat_features=cat_cols)

        cb_model = CatBoostRegressor(iterations=20000, eval_metric='RMSE', loss_function='RMSE', verbose = 0)

        cb_model.fit(cb_dtrain, eval_set=cb_dvalid, early_stopping_rounds=100, verbose_eval=1000, use_best_model=True)
        cb_models[step] = cb_model

        step += 1
        
    # cb 성능종합
    performance = [cb_models[step].best_score_['validation']['RMSE'] for step in range(1,6)]
    performance.append(np.mean(performance))

    output = pd.DataFrame({'cb':performance}, index=['cv1','cv2','cv3','cv4','cv5','평균'])
    return output

In [12]:
def knn_model(data_x, data_y):

    # knn 모델링
    performance = []
    cv = KFold(n_splits = n_splits, shuffle = True, random_state=42)

    for tr_idx, val_idx in cv.split(data_x):  

        X_train = data_x.iloc[tr_idx, :]
        y_train = data_y[tr_idx]

        X_valid = data_x.iloc[val_idx, :]
        y_valid = data_y[val_idx]

        knn_model = KNeighborsRegressor()
        knn_model.fit(X_train, y_train)

        pred_valid = knn_model.predict(X_valid)
        rmse = mean_squared_error(y_valid, pred_valid, squared=False)
        performance.append(rmse)
        
    # knn 성능종합
    performance.append(np.mean(performance))

    output = pd.DataFrame({'knn':performance}, index=['cv1','cv2','cv3','cv4','cv5','평균'])
    return output

In [13]:
def regression_model(data_x, data_y):

    # regression 모델링
    performance = []
    cv = KFold(n_splits = n_splits, shuffle = True, random_state=42)

    for tr_idx, val_idx in cv.split(data_x):  

        X_train = data_x.iloc[tr_idx, :]
        y_train = data_y[tr_idx]

        X_valid = data_x.iloc[val_idx, :]
        y_valid = data_y[val_idx]

        regression_model = LinearRegression()
        regression_model.fit(X_train, y_train)

        pred_valid = regression_model.predict(X_valid)
        rmse = mean_squared_error(y_valid, pred_valid, squared=False)
        performance.append(rmse)
        
    # regression 성능종합
    performance.append(np.mean(performance))

    output = pd.DataFrame({'regression':performance}, index=['cv1','cv2','cv3','cv4','cv5','평균'])
    return output

# train, test분할 후 모델링

#### 1. 타자

In [14]:
# train, test 분할
col_dict, X, y = Xy_split(hitter)
X, X_test, y, y_test = train_test_split(X, y, test_size = 0.25, shuffle = True, random_state = 42)

X = X.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y = y.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [15]:
# 타자데이터 도출
hitter_performance = pd.concat([rf_model(X, y),
                                  lgbm_model(X, y),
                                  xgb_model(X, y),
                                  cb_model(X, y),
                                  knn_model(X, y),
                                  regression_model(X, y)], axis=1)



 ============================ 1 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[65]	valid_0's rmse: 13235.6


 ============================ 2 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[68]	valid_0's rmse: 11654.5


 ============================ 3 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[17]	valid_0's rmse: 10671.4


 ============================ 4 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[41]	valid_0's rmse: 8711.64


 ============================ 5 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[548]	valid_0's rmse: 10337
[0]	train-rmse:19384.68750	eval-rmse:21989.09766
Multipl

#### 2. 투수

In [16]:
# train, test 분할
col_dict, X, y = Xy_split(pitcher)
X, X_test, y, y_test = train_test_split(X, y, test_size = 0.25, shuffle = True, random_state = 42)

X = X.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y = y.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [17]:
# 타자데이터 도출
pitcher_performance = pd.concat([rf_model(X, y),
                                  lgbm_model(X, y),
                                  xgb_model(X, y),
                                  cb_model(X, y),
                                  knn_model(X, y),
                                  regression_model(X, y)], axis=1)



 ============================ 1 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[74]	valid_0's rmse: 7762.88


 ============================ 2 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[539]	valid_0's rmse: 9313.5


 ============================ 3 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[548]	valid_0's rmse: 9461.63


 ============================ 4 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[94]	valid_0's rmse: 8862.14


 ============================ 5 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[240]	valid_0's rmse: 7138.1
[0]	train-rmse:14789.79492	eval-rmse:13335.92969
Multi

In [18]:
hitter_performance

,rf,lgbm,xgb,cb,knn,regression
cv1,13561.807245,13235.567673,13832.114258,12236.072619,17249.743205,13770.395030
cv2,11455.233712,11654.543109,12093.037109,10926.138209,17061.099699,15272.203281
cv3,10082.990099,10671.438620,10006.703125,10020.769856,12510.450248,12216.848866
cv4,9628.409984,8711.636553,9658.063477,8315.519599,14320.345788,12840.131767
cv5,11197.716288,10337.035438,11252.265625,10517.642820,16625.205272,13679.533944
평균,11185.231466,10922.044279,11368.436719,10403.228621,15553.368842,13555.822578


In [19]:
pitcher_performance

,rf,lgbm,xgb,cb,knn,regression
cv1,8447.357246,7762.879845,9459.467773,7267.955018,10875.977636,10068.413457
cv2,10390.678561,9313.500569,10899.259766,10227.098194,13236.532193,13148.466574
cv3,10679.124535,9461.634384,11342.643555,9827.883056,13201.556634,12504.744203
cv4,9234.140039,8862.140299,8306.385742,8897.496598,12274.837812,11932.387047
cv5,8588.781752,7138.095401,8030.754395,7710.327394,10667.110822,11642.990525
평균,9468.016427,8507.650100,9607.702246,8786.152052,12051.203019,11859.400361
